In [ ]:
import pandas as pd

def prepare_data_get_tblfreq(
    p_df,                            # O DataFrame de entrada
    p_colname,                       # O nome da coluna para analisar
    p_valuecounts_labels=None,       # Nomes desejados para as colunas da tabela final, ex: ['Categoria', 'Frequencia']
    perc_col_name='Porcentagem_Valida (%)' # Nome para a nova coluna de porcentagem
):
    """
    Prepara uma tabela de frequência para uma coluna especificada, incluindo porcentagens válidas.

    Args:
        p_df (pd.DataFrame): O DataFrame de entrada.
        p_colname (str): O nome da coluna para a qual a frequência será calculada.
        p_valuecounts_labels (list, optional): Lista de duas strings para nomear as colunas
                                               da tabela de frequência resultante (categoria, contagem).
                                               Defaults to None, que usará nomes padrão.
        perc_col_name (str, optional): Nome para a coluna de porcentagem válida.
                                       Defaults to 'Porcentagem_Valida (%)'.

    Returns:
        pd.DataFrame: Uma tabela de frequência com colunas para categoria, contagem e porcentagem válida.
                      Retorna None se a coluna especificada não existir ou não tiver dados válidos.
    """
    if p_colname not in p_df.columns:
        print(f"Erro: A coluna '{p_colname}' não existe no DataFrame fornecido.")
        return None

    # 1. Obtem as contagens de valores para a coluna especificada (NaNs são ignorados por padrão)
    s_value_counts = p_df[p_colname].value_counts(dropna=True)

    # --- LÓGICA SIMPLIFICADA AQUI ---
    if s_value_counts.empty:
        print(f"Aviso: A coluna '{p_colname}' não possui dados válidos para contagem.")

    # 1. Definir os nomes base para as colunas de categoria e frequência
    if p_valuecounts_labels and len(p_valuecounts_labels) == 2:
       nomes_base_colunas = p_valuecounts_labels
    else:
       nomes_base_colunas = ['Categoria', 'Frequencia']

    # 2. Adicionar o nome da coluna de porcentagem à lista de nomes base
    nomes_finais_colunas = nomes_base_colunas + [perc_col_name]

    # 3. Retornar o DataFrame vazio com os nomes de colunas definidos
    return pd.DataFrame(columns=nomes_finais_colunas)

    # 2. Converte a Série de contagens para um DataFrame
    df_value_counts = s_value_counts.reset_index()

    # 3. Atribui os nomes às colunas do df_value_counts
    # A primeira coluna é a categoria, a segunda é a contagem.
    if p_valuecounts_labels and len(p_valuecounts_labels) == 2:
        df_value_counts.columns = p_valuecounts_labels
    else:
        # Nomes padrão se p_valuecounts_labels não for fornecido ou for inválido
        # Tenta usar o nome do índice da série original para a coluna de categoria
        category_col_default_name = s_value_counts.index.name if s_value_counts.index.name is not None else p_colname
        # O nome da série value_counts é frequentemente 'count' ou o nome da coluna original
        count_col_default_name = s_value_counts.name if s_value_counts.name is not None else 'Frequencia'
        df_value_counts.columns = [category_col_default_name, count_col_default_name]

    # Identifica o nome da coluna de contagem (é a segunda coluna)
    count_column_name = df_value_counts.columns[1]

    # 4. Calcula o total de observações válidas
    n_valid = df_value_counts[count_column_name].sum()

    # 5. Calcula a Série de porcentagens válidas
    # Esta é a linha que você queria completar, adaptada para usar count_column_name
    perc_valid_series = (df_value_counts[count_column_name] / n_valid) * 100

    # 6. Adiciona a Série de porcentagens como uma nova coluna ao DataFrame
    df_value_counts = df_value_counts.assign(**{perc_col_name: perc_valid_series})

    return df_value_counts

# Exemplo de como usar a função revisada:
# Supondo que você tenha um DataFrame 'meu_df' e queira analisar a coluna 'status_civil'
# dados_exemplo = {'status_civil': ['Solteiro', 'Casado', 'Solteiro', 'Divorciado', 'Casado', 'Solteiro', None, 'Viúvo']}
# meu_df = pd.DataFrame(dados_exemplo)

# tabela_status_civil = prepare_data_get_tblfreq(
#     meu_df,
#     'status_civil',
#     p_valuecounts_labels=['Estado Civil', 'Nº de Pessoas'],
#     perc_col_name='Percentual Válido (%)'
# )

# if tabela_status_civil is not None:
#     print(tabela_status_civil)

In [19]:
def prepare_data_get_tblfreq(
    p_df,
    p_colname,
    p_valuecounts_labels):

    #obtem o df_value_counts para a coluna espedificada
    df_value_counts = df[p_colname].value_counts().reset_index()
    #calcula alguns totais importantes para a tbl de frequencia
    n_valid = df_value_counts['count'].sum()
    perc_valid_serie = ((df_value_counts['count'] / n_valid) * 100).round(2)

    #atribui os lavels pra cada categoria do value counts
    df_value_counts.columns = p_valuecounts_labels    
    
    df_value_counts = df_value_counts.assign(perc = perc_valid_serie)
    return df_value_counts


In [23]:
import plotly.express as px

def plotly__bar(
    p_df,
    p_colname,
    p_title,
    p_valuecounts_labels,
    pHtmlExportFilename):
    
    df_tblfreq = prepare_data_get_tblfreq(p_df, p_colname, p_valuecounts_labels);
    print(df_tblfreq)
    
    fig = px.bar(
        df_tblfreq, 
        x = p_valuecounts_labels[0], 
        y = p_valuecounts_labels[1],
        title=p_title,
        text='perc',
        labels={"px_colname": "px_label", "py_colname": "py_label"})
    
    text_template = '%{text:.2f}%' # Formata para duas casas decimais e adiciona %
    
    # Atualiza os traços para formatar e posicionar o texto
    # texttemplate define como o texto do parâmetro 'text' será exibido
    # textposition='outside' coloca o texto fora da barra (pode ser 'inside', 'auto')
    fig.update_traces(texttemplate=text_template, textposition='outside')
    
    fig.write_html(pHtmlExportFilename, include_plotlyjs='inline')    